# Using the Platform's Control Plane API

The platform's Control Plane provides access to the cluster-management using REST APIs. Using CRUD-like http client, you will be able to get, list, create, update and delete resources and trigger operations against the Iguazio System.


## Initialize

In [ ]:
import v3io.controlplane

Create a control plane client

In [ ]:
client = v3io.controlplane.Client(endpoint="dashboard.default-tenant.app.iguazio.com",
                                  username="some-admin",
                                  access_key="some-access-id")

>
> **Note**: If no endpoint or username arguments are given, both would be resolved from OS environment variables.
> `username` from `V3IO_USERNAME` and `endpoint` from `V3IO_API`.
>
> **Note**: You can also use your password to authenticate against the management API, e.g.:
>
```python
import v3io.controlplane
client = v3io.controlplane.Client(endpoint='https://dashboard.default-tenant.app.mysys.iguazio.com/',
                                  username="someone",
                                  password="my-pass")
await client.login()
```
>
> But know that you would be required to log in immediately after.
>
> **Note**: You can get the access key from the platform dashboard: select the user-profile picture or icon from the top right corner of any page, and select **Access Keys** from the menu. In the **Access Keys** window, either copy an existing access key or create a new
> key (*Make sure you assign it with control plane) and copy it.
>
> When running python code on local machine that connects to a remote Iguazio platform, you can obtain the URL of your cluster by copying the API URL.


## RESTful API

Most of the resources support all CRUD methods. Some endpoints within the management API are considered "operations", which means
the response may not represent a resource. Some operations are long-living, which means a background job is created whereas the response will hold its id.

To differentiate between a long-living task to a immediate action, it is either up to the user to validate the response and see if it holds a job_id, or, using the SDK, to indicate whether it should wait for the job to be finished.

To create a resource, we will use the `crud` module and its `.create()` method while providing the mandatory fields with it.

### Users API
#### Creating a user

An example of creating a user with default assigned policies (Developer, Application Read Only). to set different policies,
override the `create` arg - `assigned_policies` with any default of `v3io.controlplane.constants.TenantManagementRoles` enum.

In [ ]:
new_user = await v3io.controlplane.User.create(client,
                                               username="anewuser",
                                               password="rePlaceme12@!",
                                               email="user@iguazio.com",
                                               first_name="someone",
                                               last_name="fromthepast")
print(new_user.id)

#### Get self user
Getting the user object correlated to the HTTP Client created above.

In [ ]:
running_user = await v3io.controlplane.User.self(client)
print(f"The client's username is {running_user.username}")

#### Update user
An examples of updating a user attribute, such as its last name.

In [ ]:
new_user.last_name = "my name lastname"
new_user = await new_user.update(client)
print(f"Updated lastname: {new_user.last_name}")

#### List users
An examples of listing users

In [ ]:
# list all users
users = await v3io.controlplane.User.list(client)
for user in users:
    print(f"ID: {user.id}, First Name: {user.first_name}")

# list users, filter by their first_name attribute
some_users = await v3io.controlplane.User.list(client, filter_by={"first_name": "someone"})
print(f"ID: {some_users[0].id}")

### Change user groups
An example of adding / removing a user from a group

In [ ]:
# get or create user group
user_groups = await v3io.controlplane.UserGroup.list(client, filter_by={"name":"random-people"})
if user_groups:
    user_group = user_groups[0]
else:
    user_group = await v3io.controlplane.UserGroup.create(client, name="random-people")

# add user to group
await new_user.add_to_group(client, user_group.id)

# get the group with its users
user_group = await v3io.controlplane.UserGroup.get(client, user_group.id, include=["users"])

# make sure we see the user in user group users
found = len(list(filter(lambda user: user["id"] == new_user.id, user_group.relationships.get("users").get("data")))) > 0
print("User was added: " + str(found))

# remove user from group
await new_user.remove_from_group(client, user_group.id)

>
> **Notes**:
>  - To list resources and filtering by a field, we will use `filter_by={"fieldname": "fieldvalue"}`
>  - To include a resource related to another (e.g.: user group includes users), we will use `include=["users"]` when we `.get()` the group. This is not mandatory.
>

#### Delete user
An example of deleting a user,

In [ ]:
await new_user.delete(client, ignore_missing=False)